<a href="https://colab.research.google.com/github/apaimyshev/restoration_colabs/blob/main/chistogipoteticheski_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="black" size="+3">Набор нейросетевых инструментов для реставрации видео</font></b>

Создан на базе блокнота ENTAR Rus от [@tg-bomze](https://github.com/tg-bomze).

Создал Антон Паймышев, tg: [@chistogipoteticheski](https://t.me/chistogipoteticheski).  
21 ноября 2020 года.

# <b>Подготовка к работе

In [1]:
#@title ##**Устанавливаем необходимые библиотеки** { display-mode: "form" }


!pip install youtube_dl
!pip install ffmpeg
!pip install ffmpeg-python
!pip install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install scipy==1.4.1
!CUDA_VISIBLE_DEVICES=0
!sudo apt-get install imagemagick imagemagick-doc
!pip install gdown==3.6.4

from IPython.display import clear_output
from google.colab import files
import imageio
import youtube_dl
import cv2
import os
import torch
import fastai
import ffmpeg
import os.path as osp
import logging
import shutil
import re
import gc
from PIL import Image
from tqdm import tqdm
from os import path
import numpy as np
import moviepy.editor as mpy
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from pathlib import Path
import sys
import glob
from IPython import display as ipythondisplay
from IPython.display import Image as ipythonimage
torch.backends.cudnn.benchmark=True
import subprocess as sp

%matplotlib inline

!rm -rf sample_data
clear_output()
print("Установили все зависимости. Можно переходить к следующему шагу.")


Установили все зависимости. Можно переходить к следующему шагу.


In [5]:
#@title ##**Загружаем видео (обязательный формат MP4)** { display-mode: "form" }
#@markdown *Ниже введите ссылку на видео (например, YouTube или Twitter), либо оставьте поле **source_url** пустым (в таком случае будет предложено загрузить ролик с компьютера).*
%cd /content
source_url = '' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  f_ext = fn.split(".")[-1]
  file_name = "video." + f_ext
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    f_ext = fn.split(".")[-1]
    file_name = "video." + f_ext
    !mv -f $fn $file_name

file_back = "video_back." + f_ext

shutil.copy(file_name, file_back)

file_src = "/content/" + file_name
clear_output()
fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
print("Видео загружено, переходите к следующему шагу.")
#@markdown *Не рекомендуется загружать видео, длящиеся дольше одной минуты и содержащие в названии файла точки или пробелы.*

#@markdown *Если при выполнении возникнет ошибка, то запустите этот блок заново*

/content


Saving Памятник Александру III.mp4 to Памятник Александру III.mp4
User uploaded file "Памятник Александру III.mp4" with length 6128433 bytes
360


# <b>Удаление мелких помех ([DeepRemaster](https://github.com/satoshiiizuka/siggraphasia2019_remastering))


Автор: [Satoshi Iizuka](http://iizuka.cs.tsukuba.ac.jp/index_eng.html)

In [40]:
#@title ##**Клонируем репозиторий** { display-mode: "form" }
%cd /content
!git clone https://github.com/satoshiiizuka/siggraphasia2019_remastering.git DeepRemaster
file_remaster = "/content/DeepRemaster/" + file_name
shutil.copy(file_src, file_remaster)
%cd /content/DeepRemaster
clear_output()

In [36]:
#@title ##**Качаем предобученную модель** { display-mode: "form" }
%cd /content/DeepRemaster
!rm -rf model/remasternet.pth.tar
!bash ./download_model.sh


/content/DeepRemaster
--2020-11-21 15:08:07--  http://iizuka.cs.tsukuba.ac.jp/data/remasternet.pth.tar
Resolving iizuka.cs.tsukuba.ac.jp (iizuka.cs.tsukuba.ac.jp)... 130.158.81.20
Connecting to iizuka.cs.tsukuba.ac.jp (iizuka.cs.tsukuba.ac.jp)|130.158.81.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256796033 (245M) [application/x-tar]
Saving to: ‘model/remasternet.pth.tar’

model/remasternet.p 100%[===================>] 244.90M  19.2MB/s    in 14s     

2020-11-21 15:08:21 (17.3 MB/s) - ‘model/remasternet.pth.tar’ saved [256796033/256796033]

Checking integrity (md5sum)...
Download finished successfully!


In [41]:
#@title ##**Удаляем помехи на кадрах** { display-mode: "form" }
FRAME_WIDTH = int(cv2.VideoCapture(file_remaster).get(cv2.CAP_PROP_FRAME_WIDTH))
FRAME_HEIGHT = int(cv2.VideoCapture(file_remaster).get(cv2.CAP_PROP_FRAME_HEIGHT))
mindim = min(FRAME_WIDTH, FRAME_HEIGHT)

print(mindim)

!python remaster.py --input $file_remaster --disable_colorization --gpu --mindim $mindim
clear_output()

352


In [38]:
#@title ##**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
!rm -rf $file_src
!cp -r video_out.mp4 $file_src
!cp -r video_out.mp4 /content/denoise_video.mp4
what_next = 'download' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/denoise_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download('/content/denoise_video.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# <b>Интерполяция кадров ([DAIN](https://github.com/baowenbo/DAIN))


Авторы: [Wenbo Bao](mailto:bwb0813@gmail.com); [Wei-Sheng (Jason) Lai](mailto:phoenix104104@gmail.com)

In [49]:
#@title ##**Настройка параметров** { display-mode: "form" }

INPUT_FILEPATH = "/content/video.mp4" 
OUTPUT_FILE_PATH = "/content/video.mp4" 
TARGET_FPS = 60 
FRAME_INPUT_DIR = '/content/DAIN/input_frames' 
FRAME_OUTPUT_DIR = '/content/DAIN/output_frames' 
START_FRAME = 1
END_FRAME = -1 
SEAMLESS = False 
RESIZE_HOTFIX = False 
AUTO_REMOVE = False 

# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

print("Настройка завершена")

name, driver_version, memory.total [MiB]
Tesla T4, 418.67, 15079 MiB
Настройка завершена


In [17]:
#@title ##**Клонируем репозиторий** { display-mode: "form" }
%cd /content
!git clone -b master --depth 1 https://github.com/baowenbo/DAIN /content/DAIN
%cd /content/DAIN
!git log -1

/content
Cloning into '/content/DAIN'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 142 (delta 42), reused 107 (delta 35), pack-reused 0
Receiving objects: 100% (142/142), 97.64 KiB | 16.27 MiB/s, done.
Resolving deltas: 100% (42/42), done.
/content/DAIN
commit 9d9c0d7b3718dfcda9061c85efec472478a3aa86 (grafted, HEAD -> master, origin/master, origin/HEAD)
Author: Wenbo Bao <baowenbo@users.noreply.github.com>
Date:   Thu Oct 29 12:16:31 2020 +0800

    Merge pull request #89 from AlphaGit/partial-video-processing
    
    Colab: Partial video processing


In [18]:
#@title ##**Собираем DAIN (~15 минут)** { display-mode: "form" }
# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd /content/DAIN/my_package/
!./build.sh
clear_output()
print("Building #1 done.")

Building #1 done.


In [19]:
#@title ##**Собираем DAIN еще немного (~5 минут)** { display-mode: "form" }
# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
clear_output()
print("Building #2 done.")

Building #2 done.


In [20]:
#@title ##**Качаем предобученную модель** { display-mode: "form" }

# Downloading pre-trained model
%cd /content/DAIN
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

/content/DAIN
--2020-11-21 14:38:47--  http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth
Resolving vllab1.ucmerced.edu (vllab1.ucmerced.edu)... 169.236.184.68
Connecting to vllab1.ucmerced.edu (vllab1.ucmerced.edu)|169.236.184.68|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96319643 (92M)
Saving to: ‘model_weights/best.pth’

model_weights/best. 100%[===================>]  91.86M  29.8MB/s    in 3.1s    

2020-11-21 14:38:50 (29.8 MB/s) - ‘model_weights/best.pth’ saved [96319643/96319643]



In [21]:
#@title ##**Определяем FPS исходного видео** { display-mode: "form" }

# Detecting FPS of input file.
%shell yes | cp -f {INPUT_FILEPATH} /content/DAIN/

filename = os.path.basename(INPUT_FILEPATH)

cap = cv2.VideoCapture(f'/content/DAIN/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Input file has {fps} fps")

if(fps/TARGET_FPS>0.5):
  print("Define a higher fps, because there is not enough time for new frames. (Old FPS)/(New FPS) should be lower than 0.5. Interpolation will fail if you try.")

Input file has 25.0 fps


In [22]:
#@title ##**Преобразуем видео для дальнейшей работы** { display-mode: "form" }

# ffmpeg extract - Generating individual frame PNGs from the source file.
%shell rm -rf '{FRAME_INPUT_DIR}'
%shell mkdir -p '{FRAME_INPUT_DIR}'

if (END_FRAME==-1):
  %shell ffmpeg -i '/content/DAIN/{filename}' -vf 'select=gte(n\,{START_FRAME}),setpts=PTS-STARTPTS' '{FRAME_INPUT_DIR}/%05d.png'
else:
  %shell ffmpeg -i '/content/DAIN/{filename}' -vf 'select=between(n\,{START_FRAME}\,{END_FRAME}),setpts=PTS-STARTPTS' '{FRAME_INPUT_DIR}/%05d.png'

clear_output()

png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
frame_count = int(png_generated_count_command_result.output.strip())

import shutil
if SEAMLESS:
  frame_count += 1
  first_frame = f"{FRAME_INPUT_DIR}/00001.png"
  new_last_frame = f"{FRAME_INPUT_DIR}/{frame_count.zfill(5)}.png"
  shutil.copyfile(first_frame, new_last_frame)

print(f"{frame_count} frame PNGs generated.")

# Checking if PNGs do have alpha
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")

# Removing alpha if detected
if "a" in channels:
  print("Alpha channel detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))


264
264 frame PNGs generated.


In [24]:
#@title ##**Интерполируем!** { display-mode: "form" }

# Interpolation
%shell mkdir -p '{FRAME_OUTPUT_DIR}'
%cd /content/DAIN

!python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {frame_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'

/content/DAIN
revise the unique id to a random numer 60774
Namespace(SAVED_MODEL=None, alpha=[0.0, 1.0], arg='./model_weights/60774-Sat-Nov-21-14-39/args.txt', batch_size=1, channels=3, ctx_lr_coe=1.0, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, depth_lr_coe=0.001, dtype=<class 'torch.cuda.FloatTensor'>, end_frame=264, epsilon=1e-06, factor=0.2, filter_lr_coe=1.0, filter_size=4, flow_lr_coe=0.01, force=False, frame_input_dir='/content/DAIN/input_frames', frame_output_dir='/content/DAIN/output_frames', log='./model_weights/60774-Sat-Nov-21-14-39/log.txt', lr=0.002, netName='DAIN_slowmotion', no_date=False, numEpoch=100, occ_lr_coe=1.0, patience=5, rectify_lr=0.001, save_path='./model_weights/60774-Sat-Nov-21-14-39', save_which=1, seed=1, start_frame=1, time_step=0.4166666666666667, uid=None, use_cuda=True, use_cudnn=1, weight_decay=0, workers=8)
cudnn is used
Interpolate 1 frames
****** Processed frame 1 | Time per frame (avg): 8.63s | Time left: 0:37:

In [25]:
#@title ##**Собираем видео** { display-mode: "form" }

# Finding DAIN Frames, upscaling and cropping to match original
%cd {FRAME_OUTPUT_DIR}

if (RESIZE_HOTFIX):
  images = []
  for filename in os.listdir(FRAME_OUTPUT_DIR):
    img = cv2.imread(os.path.join(FRAME_OUTPUT_DIR, filename))
    filename = os.path.splitext(filename)[0]
    if(not filename.endswith('0')):
      dimensions = (img.shape[1]+2, img.shape[0]+2)
      resized = cv2.resize(img, dimensions, interpolation=cv2.INTER_LANCZOS4)
      crop = resized[1:(dimensions[1]-1), 1:(dimensions[0]-1)]
      cv2.imwrite(f"{filename}.png", crop)

################
# Create output video
%cd {FRAME_OUTPUT_DIR}
#%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' '{OUTPUT_FILE_PATH}'

####################
#@title ##**Собираем кадры в видео** { display-mode: "form" }
frames = []
img = os.listdir(FRAME_OUTPUT_DIR)
img.sort()
for i in img:
  frames.append(imageio.imread(FRAME_OUTPUT_DIR + "/" + i))
frames = np.array(frames)
imageio.mimsave("/content/interpolated_video.mp4", frames, fps = TARGET_FPS)

print('Сборка завершена')
!cp -r /content/interpolated_video.mp4 /content/video.mp4
#clear_output()

/content/DAIN/output_frames


In [28]:
#@title ##**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
what_next = 'download' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/interpolated_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download('/content/interpolated_video.mp4')

if(AUTO_REMOVE):
  !rm -rf {FRAME_OUTPUT_DIR}/*



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# <b>Увеличение разрешения ([ESRGAN](https://github.com/xinntao/ESRGAN))



Автор: [Xintao Wang](https://xinntao.github.io)

In [29]:
#@title ##**Клонируем репозиторий и качаем необходимые компоненты** { display-mode: "form" }

%cd /content
!git clone https://github.com/xinntao/ESRGAN.git
!cp -r video.mp4 /content/ESRGAN/
%cd /content/ESRGAN
!git checkout tags/old-arch

%cd /content/ESRGAN/models/
my_file_id = "1MJFgqXJrMkPdKtiuy7C6xfsU1QIbXEb-"
!gdown https://drive.google.com/uc?id={my_file_id}


%cd /content/ESRGAN

import architecture as arch
import os.path

!mkdir frames
!rm -rf results/baboon_ESRGAN.png
clear_output()



In [30]:
#@title ##**Разделяем видео на кадры** { display-mode: "form" }
frames_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
fps_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FPS))
vidcap = cv2.VideoCapture('video.mp4')
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

In [31]:
#@title ##**Увеличиваем разрешение кадра** { display-mode: "form" }
#@markdown **Во сколько раз увеличить разрешение:**
upscale = 4 #@param {type: "slider", min: 4, max: 8}
%env CUDA_VISIBLE_DEVICES=0
device = torch.device('cuda')
model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=upscale, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
model.load_state_dict(torch.load('models/{:s}'.format('RRDB_ESRGAN_x4_old_arch.pth')), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)

count_frames = 0

for path in glob.glob('frames/*'):
    base = os.path.splitext(os.path.basename(path))[0]
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    path = 'results/{:s}_rlt.png'.format(base)
    cv2.imwrite(path, output)
    count_frames += 1
    clear_output()
    print("Обработано: {} из {}".format(str(count_frames), str(frames_of_video)))
clear_output()

In [32]:
#@title ##**Собираем кадры в видео** { display-mode: "form" }
%cd /content/ESRGAN/
frames = []
img = os.listdir("results/")
img.sort()
for i in img:
  frames.append(imageio.imread("results/"+i))
frames = np.array(frames)
imageio.mimsave("upscaled_video.mp4", frames, fps=fps_of_video)

print('Сборка завершена')
!cp -r upscaled_video.mp4 /content/upscaled_video.mp4
!cp -r upscaled_video.mp4 /content/video.mp4
#clear_output()

/content/ESRGAN
Сборка завершена


In [33]:
#@title ##**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
what_next = 'download' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/upscaled_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("/content/upscaled_video.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# <b>Раскрашивание видео ([DeOldify](https://github.com/jantic/DeOldify))


Автор: [Jason Antic](https://twitter.com/citnaj)

In [42]:
#@title ##**Клонируем репозиторий и устанавливаем необходимые зависимости** { display-mode: "form" }
%cd /content
!git clone https://github.com/jantic/DeOldify.git
%cd /content/DeOldify
!pip install -r colab_requirements.txt
clear_output()

In [43]:
#@title ##**Качаем предобученную модель** { display-mode: "form" }
from deoldify.visualize import *
!mkdir 'models'
!rm -rf models/ColorizeVideo_gen.pth
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth
colorizer = get_video_colorizer()
clear_output()

In [44]:
#@title ##**Раскрашиваем видео** { display-mode: "form" }
#@markdown **21, 35** *- оптимальные показатели*
!rm -rf video
!mkdir 'video'
!mkdir 'video/source'
!cp -r $file_src video/source/video.mp4

render_factor = 21  #@param {type: "slider", min: 5, max: 44}
video_path = colorizer.colorize_from_file_name('video.mp4', render_factor)
clear_output()

In [48]:
#@title ##**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
!cp -r video/result/video.mp4 $file_src
!cp -r video/result/video.mp4 /content/colored_video.mp4

what_next = 'download' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display(file_src, height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("/content/colored_video.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>